### Installs used packages

In [ ]:
!pip install selenium
!pip install pandas
!pip install numpy

### Imports used packages

In [1]:
import time
import json
import pandas as pd
import numpy as np

### Removes DataFrame size limits

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Credit Suisse IPOs

### Webscrapes Fidelity.com

In [ ]:
from selenium import webdriver

PATH = '/home/jp/Python/chromedriver'
driver = webdriver.Chrome(PATH)
url = 'https://www.fidelity.com/stock-trading/previous-year-ipos'

driver.get(url)
driver.maximize_window()

ipos = []

for year in range(1996,2021):
    table = driver.find_element_by_xpath("//*[@id='{}']/table".format(year))

    for row in table.find_elements_by_xpath(".//tr"):
        ipo = [td.text for td in row.find_elements_by_xpath(".//td")]
        if year < 2016:
            if (ipo != []) and (ipo[5] == 'IPO' ) and ('Credit Suisse' in ipo[6]):
                ipos.append(ipo)
        elif year > 2015:
            if (ipo != []) and (ipo[4] == 'IPO' ) and ('Credit Suisse' in ipo[5]):
                ipos.append(ipo)
            
with open('fidelity-credit_suisse-IPOs.json', 'w+') as f:
    json.dump(ipos, f)

driver.close()

### Creates DataFrame of Fidelity IPOs

In [16]:
with open('fidelity-credit_suisse-IPOs.json', 'rb') as f:
    data = json.load(f)

ipos = {
    'Date':[],
    'Issuer':[],
    'Symbol':[],
    'Managers':[],
    'Access Bookmaker':[],
    'Offer Price':[]
}

for ipo in data:
    ipos['Issuer'].append(ipo[0])
    ipos['Date'].append(ipo[1])
    ipos['Offer Price'].append(float(ipo[2][1:]))
    ipos['Symbol'].append(ipo[3].strip())
    if int(ipo[1][-2:]) > 15:
        ipos['Managers'].append(ipo[5])
        ipos['Access Bookmaker'].append(ipo[6])
        
    else:
        ipos['Managers'].append(ipo[6])
        ipos['Access Bookmaker'].append("Nan")
    
fid = pd.DataFrame(ipos)

fid

Date                              Issuer Symbol  \
0    11/16/2006                              Venoco     VQ   
1    11/09/2006                ACA Capital Holdings    ACA   
2    09/20/2006                     Warner Chilcott   WCRX   
3    08/08/2006                          Qimonda AG     QI   
4    05/16/2006                    Penson Worldwide   PNSN   
5    11/14/2007                    Energy Solutions     ES   
6    11/06/2007                     BioForm Medical   BFRM   
7    08/08/2007                     WuXi PharmaTech     WX   
8    02/07/2007                  National CineMedia   NCMI   
9    02/07/2007               Mellanox Technologies   MLNX   
10   05/08/2008             Western Gas Partners LP    WES   
11   01/28/2008                           Cascal NV    HOO   
12   12/15/2009         Cobalt International Energy    CIE   
13   07/29/2009  PennyMac Mortgage Investment Trust    PMT   
14   12/01/2010                                FXCM   FXCM   
15   08/11/2010                            RealPage     RP   
16   08/05/2010             NXP Semiconductors N.V.   NXPI   
17   08/05/2010                 IntraLinks Holdings     IL   
18   07/14/2010                  Smart Technologies    SMT   
19   03/30/2010          SS&C Technologies Holdings   SSNC   
20   11/03/2011                             Groupon   GRPN   
21   08/16/2011              Tudou Holdings Limited   TUDO   
22   07/20/2011                      SunCoke Energy    SXC   
23   06/23/2011                                KiOR   KIOR   
24   06/08/2011             Taomee Holdings Limited   TAOM   
25   05/25/2011    Freescale Semiconductor Holdings    FSL   
26   05/03/2011                     Boingo Wireless   WIFI   
27   04/19/2011                 Tesoro Logistics LP   TLLP   
28   03/29/2011            Apollo Global Management    APO   
29   01/25/2011                    Nielsen Holdings   NLSN   
30   12/12/2012                          PBF Energy    PBF   
31   10/25/2012         The WhiteWave Foods Company   WWAV   
32   10/11/2012                  Diamondback Energy   FANG   
33   10/02/2012         JAVELIN Mortgage Investment    JMI   
34   08/09/2012                   Manchester United   MANU   
35   08/09/2012                Performant Financial   PFMT   
36   05/09/2012                            Audience   ADNC   
37   05/02/2012                The Carlyle Group LP     CG   
38   04/19/2012                          Proofpoint   PFPT   
39   04/11/2012           Forum Energy Technologies    FET   
40   03/21/2012                              Vantiv   VNTV   
41   03/14/2012       Allison Transmission Holdings   ALSN   
42   02/23/2012                         Bazaarvoice     BV   
43   02/07/2012               Caesars Entertainment    CZR   
44   11/20/2013                        Evogene Ltd.   EVGN   
45   11/12/2013             Dynagas LNG Partners LP   DLNG   
46   10/24/2013       Endurance International Group   EIGI   
47   07/31/2013              Sprouts Farmers Market    SFM   
48   05/22/2013                    Ply Gem Holdings   PGEM   
49   04/17/2013                             Taminco    TAM   
50   04/09/2013                Taylor Morrison Home   TMHC   
51   02/07/2013                      ZAIS Financial    ZFC   
52   12/11/2014                             Workiva     WK   
53   12/11/2014                         Hortonworks    HDP   
54   12/10/2014                                Momo   MOMO   
55   12/10/2014                         LendingClub     LC   
56   11/17/2014                       STORE Capital   STOR   
57   11/06/2014               INC Research Holdings   INCR   
58   11/06/2014                            Freshpet   FRPT   
59   10/29/2014       Fifth Street Asset Management   FSAM   
60   10/09/2014                   Diplomat Pharmacy   DPLO   
61   10/08/2014         OM Asset Management Limited   OMAM   
62   10/08/2014                     USD Partners LP   USDP   
63   10/02/2014                            

### Create DataFrame from IPO Scoop XLS

In [4]:
file = 'ipos_excel.csv'
scoop = pd.read_csv(file)
scoop['1st Day % Chg'] = ((scoop['1st Day Close'] - scoop['Offer Price']) / scoop['Offer Price'])
scoop.replace([np.inf, -np.inf], np.nan, inplace=True)
scoop.dropna(inplace=True)

### IPOs that are on Fidelity site but not IPO Scoop

In [5]:
fid[fid['Symbol'].isin(scoop['Symbol'])==False]

Date                     Issuer Symbol  \
29   01/25/2011           Nielsen Holdings   NLSN   
44   11/20/2013               Evogene Ltd.   EVGN   
81   11/12/2015                  Mesoblast   MESO   
138  09/28/2017          PQ Group Holdings    PQG   
153  01/26/2017           Jeld-Wen Holding   JELD   
154  01/26/2017         Jagged Peak Energy    JAG   
155  01/25/2017                     ObsEva   OBSV   
204  03/07/2019              Futu Holdings    FHL   
206  12/15/2020                    BioAtla   BCAB   
207  12/10/2020                     Airbnb   ABNB   
208  12/10/2020                    Certara   CERT   
209  11/19/2020              Sotera Health    SHC   
210  10/28/2020       Allegro MicroSystems   ALGM   
211  10/28/2020                    Galecto   GLTO   
212  10/27/2020                 MediaAlpha    MAX   
213  10/21/2020              Datto Holding    MSP   
214  10/08/2020                     iHuman     IH   
215  10/08/2020         Spruce Biosciences   SPRB   
216  10/01/2020  Academy Sports & Outdoors    ASO   
217  09/22/2020             Corsair Gaming   CRSR   
218  09/17/2020             Unity Software      U   
219  09/16/2020           Pactiv Evergreen   PTVE   
222  08/18/2020                    Inhibrx   INBX   

                                                                                                                                                                             Managers  \
29                                                                                                  JP Morgan, Morgan Stanley, Credit Suisse, Deutsche Bank, Goldman Sachs, Citigroup   
44                                                                                                                                                       Credit Suisse, Deutsche Bank   
81                                                                                                                                                           JP Morgan, Credit Suisse   
138                                                                                                                           Morgan Stanley, Goldman Sachs, Citigroup, Credit Suisse   
153                                                                                                                                     Barclays, Citigroup, Credit Suisse, JP Morgan   
154                                                                                                                                               Citigroup, Credit Suisse, JP Morgan   
155                                                                                                                                        Credit Suisse, Jefferies, Leerink Partners   
204                                                                                                                                                 Goldman Sachs, UBS, Credit Suisse   
206                                                                                                                                                JP Morgan, Jeffries, Credit Suisse   
207  Morgan Stanley, Goldman Sachs, Allen & Company LLC, BofA Merrill Lynch, Barclays, Citigroup, Credit Suisse, BNP Paribas, Mizuho Securities, Deutsche Bank, Jeffries, Wells Fargo   
208                                                                                              Jeffries, Morgan Stanley, BofA Merrill Lynch, Credit Suisse, Barclays, William Blair   
209                                                                                                                                 JP Morgan, Credit Suisse, Goldman Sachs, Jeffries   
210                                                                                                                                              Barclays, Credit Suisse, Wells Fargo   
211                                                                                                                                    BofA Merrill 

### Fidelity IPOs with Open and Close Data from Scoop
Note: There are duplicates?

In [18]:
fid_ipos = scoop[scoop['Symbol'].isin(fid['Symbol'])==True]
fid_ipos = fid_ipos[fid_ipos['Managers'].str.contains('Credit Suisse')==True]

### Credit Suisse IPOs that are on IPO Scoop but not Fidelity 
Note: Some rows on IPO Scoop excel do not list Credit Suisse as manager however said IPO is on Fidelity site


In [19]:
scoop_only = scoop[scoop['Symbol'].isin(fid['Symbol'])==False]
scoop_only = scoop_only[scoop_only['Managers'].str.contains('Credit Suisse')==True]
scoop_only = scoop_only[scoop_only['Symbol'].str.contains('.U')==False]
scoop_only = scoop_only[scoop_only['Issuer'].str.contains('Acquisition')==False]
scoop_only

Date                                   Issuer Symbol  \
1      1/17/20                                   LIZHI    LIZI   
15      2/6/20                                     PPD     PPD   
176    8/19/20                                 Inhibrx    IBNX   
321    7/18/19            AssetMark Financial Holdings     AMK   
327    7/18/19                        Owl Rock Capital    ORCC   
337    7/25/19            Sunnova Energy International    NOVA   
356    9/12/19                         SmileDirectClub     SDC   
418   12/12/19                           EHang Holdings     EH   
480    4/26/18                     Ceridian HCM Holding   CDAY   
500    5/24/18                                GreenSky    GSKY   
552    7/25/18                            Bloom Energy      BE   
637   11/15/18          Bain Capital Specialty Finance    BCSF   
638   11/15/18                             Weidai Ltd.     WEI   
658     2/1/17                        Invitation Homes    INVH   
670    3/15/17                      Ardagh Group S.A.      ARD   
707    5/12/17                 Gardner Denver Holdings     GDI   
741    7/20/17                                  Calyxt    CLXT   
774    9/29/17                                PQ Group     PGQ   
788   10/26/17                BP Midstream Partners LP    BPMP   
886     8/4/16                          First Hawaiian     FHB   
994     5/1/15              Black Stone Minerals, L.P.     BSM   
1040    7/1/15                  CNX Coal Resources LP     CNXC   
1046   7/16/15                          Jupai Holdings      JP   
1091  10/14/15                            First Data       FDC   
1120   1/23/14        Santander Consumer USA Holdings       SC   
1128   1/31/14                  New Home Company (The)    NWHM   
1152   2/27/14                             Lumenis Ltd.   LMNS   
1154    3/6/14          TriplePoint Venture Growth BDC    TPVG   
1158   3/12/14                                Achaogen    AKAO   
1163   3/20/14                     Akebia Therapeutics    AKBA   
1164   3/20/14                        MediWound Ltd.      MDWD   
1168   3/21/14                              Borderfree    BRDR   
1182   3/28/14                        Everyday Health     EVDY   
1192    4/9/14                     La Quinta Holdings       LQ   
1193   4/10/14                  Adamas Pharmaceuticals    ADMS   
1204   4/16/14                              Opus Bank      OPB   
1205   4/16/14               TriVascular Technologies     TRIV   
1209   4/17/14                         Vital Therapies     VTL   
1210   4/17/14                                    Weibo     WB   
1217    5/8/14               Alder BioPharmaceuticals     ALDR   
1218    5/8/14                         Cheetah Mobile     CMCM   
1224   5/15/14                                 Zendesk     ZEN   
1225   5/16/14            Jumei International Holding     JMEI   
1231   5/23/14                          Parsley Energy      PE   
1238   6/12/14                             Zhaopin Ltd    ZPIN   
1246   6/18/14                Viper Energy Partners LP    VNOM   
1247   6/18/14                              ZS Pharma     ZSPH   
1252   6/20/14                            Kite Pharma     KITE   
1256   6/25/14                          Materialise NV    MTLS   
1272   7/18/14                           Globant S.A.     GLOB   
1297   7/31/14                   Synchrony Financial       SYF   
1306    8/7/14                    iDreamSky Technology    DSKY   
1343   10/9/14                            MOL Global      MOLG   
1362   11/6/14                     Coherus BioSciences    CHRS   
1369  11/12/14            Axalta Coating Systems Ltd      AXTA   
1371  11/13/14   Navios Maritime Midstream Partners LP     NAP   
1372  11/13/14                    PRA Health Sciences     PRAH   
1381  11/20/14                             Cnova N.V.      CNV   
1394  12/12/14        Metaldyne Performance Group (MPG)    MPG   
1402   1/17/13                        CVR Refining

In [22]:
scoop_only.to_csv('Credit_Suisee_IPOs_not_on_Fidelity.csv')
fid_ipos.to_csv('Credit_Suisse_IPOs_on_Fidelity.csv')

### Mean of 1st Day Close % Chg of IPOs

In [21]:
non_fid_return = round(np.asarray(scoop_only['1st Day % Chg'], dtype=np.float).mean()*100,3)
fid_return = round(np.asarray(fid_ipos['1st Day % Chg'], dtype=np.float).mean()*100,3)
non_fid_return, fid_return

(16.752, 14.719)

# ALL IPOs (Not just Credit Suisse)

### Webscrapes Fidelity for all IPOs

In [ ]:
from selenium import webdriver

PATH = '/home/jp/Python/chromedriver'
driver = webdriver.Chrome(PATH)
url = 'https://www.fidelity.com/stock-trading/previous-year-ipos'

driver.get(url)
driver.maximize_window()

ipos = []

for year in range(1996,2021):
    table = driver.find_element_by_xpath("//*[@id='{}']/table".format(year))

    for row in table.find_elements_by_xpath(".//tr"):
        ipo = [td.text for td in row.find_elements_by_xpath(".//td")]
        if year < 2016:
            if (ipo != []) and (ipo[5] == 'IPO' ):
                ipos.append(ipo)
        elif year > 2015:
            if (ipo != []) and (ipo[4] == 'IPO' ):
                ipos.append(ipo)
            
with open('fidelity-IPOs.json', 'w+') as f:
    json.dump(ipos, f)

driver.close()

### Creates DataFrame

In [9]:
with open('fidelity-IPOs.json', 'rb') as f:
    data = json.load(f)

ipos = {
    'Date':[],
    'Issuer':[],
    'Symbol':[],
    'Managers':[],
    'Offer Price':[]
}

for ipo in data:
    ipos['Issuer'].append(ipo[0])
    ipos['Date'].append(ipo[1])
    ipos['Offer Price'].append(float(ipo[2][1:]))
    ipos['Symbol'].append(ipo[3].strip())
    if int(ipo[1][-2:]) > 15:
        ipos['Managers'].append(ipo[5])
        
    else:
        ipos['Managers'].append(ipo[6])
    
fid_all = pd.DataFrame(ipos)

In [10]:
all_fid_ipos = scoop[scoop['Symbol'].isin(fid_all['Symbol'])==True]
all_fid_ipos.head()

Date                       Issuer Symbol  \
0   1/17/20                       I-Mab    IMAB   
1   1/17/20                       LIZHI    LIZI   
2   1/17/20  Phoenix Tree Holdings Ltd.     DNK   
10  1/31/20      Arcutis Biotherapeutics   ARQT   
11  1/31/20  Reynolds Consumer Products    REYN   

                                       Managers  Offer Price  Opening Price  \
0                               Jefferies/ CICC         14.0          14.75   
1                      Credit Suisse/ Citigroup         11.0          11.03   
2         Citigroup/ Credit Suisse/ J.P. Morgan         13.5          13.50   
10  Goldman Sachs/ Cowen/ Guggenheim Securities         17.0          23.05   
11   Credit Suisse/ Goldman Sachs/ J.P. Morgan          26.0          27.50   

    1st Day Close  1st Day % Chg  
0           12.75      -0.089286  
1           11.63       0.057273  
2           13.50       0.000000  
10          21.80       0.282353  
11          28.55       0.098077

In [13]:
scoop_only_all = scoop[scoop['Symbol'].isin(fid_all['Symbol'])==False]
scoop_only_all = scoop_only_all[scoop_only_all['Symbol'].str.contains('.U')==False]
scoop_only_all = scoop_only_all[scoop_only_all['Issuer'].str.contains('Acquisition')==False]
scoop_only_all

Date                                                 Issuer  \
3      1/17/20                                    Velocity Financial    
6      1/28/20                             AnPac Bio-Medical Science    
7      1/29/20                                           Annovis Bio    
8      1/29/20                            Black Diamond Therapeutics    
9      1/31/20                                      1Life Healthcare    
13      2/6/20                                     Beam Therapeutics    
16      2/6/20                                           Schrodinger    
17      2/7/20                          NexPoint Real Estate Finance    
19      2/7/20                                  Professional Holding    
24     2/13/20                                          Muscle Maker    
25     2/13/20                                  Revolution Medicines    
30     2/24/20                                             ZHONGCHAO    
31     2/28/20                                           Passage BIO    
36     3/12/20                                                 IMARA    
37      4/1/20                                   WiMi Hologram Cloud    
43     4/24/20                                  ORIC Pharmaceuticals    
47      5/1/20                                     Lyra Therapeutics    
56     5/15/20                           ADC Therapeutics America SA    
61     5/22/20                                         Inari Medical    
63      6/3/20                                   Pliant Therapeutics    
66      6/5/20                     Applied Molecular Transport (AMTI)   
68      6/5/20                                    Dada Nexus Limited    
69      6/5/20                                        Legend Biotech    
73      6/9/20                                                 Vroom    
74     6/10/20                                      uCloudlink Group    
75     6/11/20                                        Lantern Pharma    
78     6/12/20                             Burning Rock Biotech Ltd.    
79     6/12/20                                         Generation Bio   
80     6/12/20                                               Vaxcyte    
85     6/19/20                                              Progenity   
86     6/19/20                                   Repare Therapeutics    
88     6/26/20                                                 Agora    
91     6/26/20                          Ebang International Holdings    
101     7/2/20                                              Lemonade    
102     7/8/20                                     BlueCity Holdings    
105    7/10/20                                  Poseida Therapeutics    
106    7/10/20                                         Quhuo Limited    
109    7/14/20                                                 nCino    
114    7/16/20                                    Relay Therapeutics    
115    7/16/20                                Trean Insurance Group     
117    7/17/20                                       Berkeley Lights    
119    7/17/20                           Pandion Therapeutics Holdco    
120    7/22/20                                          Jamf Holding    
124    7/23/20                         Montrose Environmental Group     
125    7/23/20               Skillful Craftsman Education Technology    
126    7/24/20                                               Annexon    
129    7/24/20                                        Inozyme Pharma    
130    7/24/20                                    iTeos Therapeutics    
131    7/24/20                                    Nurix Therapeutics    
133    7/29/20                                                Vertex    
134    7/30/20                                               AlloVir    
136    7/30/20                                                Li Auto   
137    7/31/20                                         1847 Goedeker    
140    7/31/20                                       Fathom Holdings

In [12]:
non_fid_return_all = round(np.asarray(scoop_only_all['1st Day % Chg'], dtype=np.float).mean()*100,3)
fid_return_all = round(np.asarray(all_fid_ipos['1st Day % Chg'], dtype=np.float).mean()*100,3)
non_fid_return_all, fid_return_all

(15.648, 13.251)